In [1]:
import sys
sys.path.insert(0, '..')

from intransparent import (
    ingest_reports_per_platform,
    long_ncmec_reports,
    compare_all_platform_reports,
    REPORTS_PER_PLATFORM,
    show,
    YEAR_LABELS,
)

data = ingest_reports_per_platform(REPORTS_PER_PLATFORM, include_redundant=True)

show('<h1>CSAM Reports per Platform</h1>')
show(f'<h2>Ranking of Social Media Firms</h2>')

ncmec = long_ncmec_reports(data)
totals = ncmec[ncmec['platform'] == 'Total']['reports']

for year in YEAR_LABELS:
    yearly = (
        ncmec.loc[year]
        .sort_values('reports', ascending=False)
        .assign(reports_pct=lambda df: df['reports'] / totals[year] * 100)
    )
    show(
        yearly,
        caption=f' Social Media by CSAM Reports File in {year}'
    )

,platform,reports,redundant,reports_pct
period,,,,
2019,Total,"16,836,694",False,100.00000000
2019,Meta,"15,884,511",False,94.34459639
2019,Alphabet,"449,283",False,2.66847518
2019,Snap,"82,030",False,0.48720966
2019,Twitter,"45,726",False,0.27158538
2019,Automattic,"10,443",False,0.06202524
2019,Pinterest,"7,360",False,0.04371405
2019,Reddit,724,False,0.00430013
2019,TikTok,596,False,0.00353989


,platform,reports,redundant,reports_pct
period,,,,
2020,Total,"21,447,786",False,100.00000000
2020,Meta,"20,307,216",False,94.68210845
2020,Alphabet,"546,704",False,2.54899970
2020,Snap,"144,095",False,0.67184091
2020,Twitter,"65,062",False,0.30335066
2020,TikTok,"22,692",False,0.10580113
2020,Automattic,"9,130",False,0.04256850
2020,Pinterest,"3,432",False,0.01600165
2020,Reddit,"2,233",False,0.01041133


,platform,reports,redundant,reports_pct
period,,,,
2021,Total,"29,157,083",False,100.0000000
2021,Meta,"26,885,302",False,92.2084764
2021,Alphabet,"875,783",False,3.0036715
2021,Snap,"512,522",False,1.7577959
2021,TikTok,"154,618",False,0.5302931
2021,Twitter,"86,666",False,0.2972382
2021,Reddit,"10,059",False,0.0344993
2021,Automattic,"4,821",False,0.0165346
2021,Pinterest,"2,283",False,0.0078300


,platform,reports,redundant,reports_pct
period,,,,
2022,Total,"31,802,525",False,100.000000
2022,Meta,"27,190,665",False,85.498447
2022,Alphabet,"2,174,548",False,6.837658
2022,Snap,"551,086",False,1.732837
2022,TikTok,"288,125",False,0.905982
2022,Twitter,"98,050",False,0.308309
2022,Reddit,"52,592",False,0.165371
2022,Pinterest,"34,310",False,0.107885
2022,Automattic,"5,035",False,0.015832


In [2]:
show('<h2>Transparency Data Quality per Social Medium</h2>')

comparisons = compare_all_platform_reports(data)

brand_holders = len(data.brands)
brand_holder_brands = sum(len(brands) for brands in data.brands.values())
# Account for '@' and 'NCMEC':
surveyed_brands = len(REPORTS_PER_PLATFORM) - 1 - 1 - brand_holders
surveyed_firms = surveyed_brands - brand_holder_brands + brand_holders
# Account for 'NCMEC':
brands_with_disclosures = len(set(data.disclosures) - set(data.brands)) - 1

show(
    f"""
    Out of {surveyed_brands} surveyed social media platforms owned by
    {surveyed_firms} firms, {brands_with_disclosures} brands make transparency
    disclosures but only {len(comparisons) + 1} platforms owned by
    {len(comparisons)} firms disclose the number of CSAM reports submitted to
    NCMEC.
    """
)

for platform, data in comparisons.items():
    show(data, caption=platform)

,pieces,reports,Δ%,NCMEC
year,,,,
2020,⋯,"3,432",0.00,"3,432"
2021,"1,608","2,684",-14.94,"2,283"
2022,"37,136","32,964",4.08,"34,310"


,pieces,reports,Δ%,NCMEC
year,,,,
2019,⋯,724,0.0,724
2020,⋯,"2,233",0.0,"2,233"
2021,"9,258","10,059",0.0,"10,059"
2022,"80,888","52,592",0.0,"52,592"


,pieces,reports,Δ%,NCMEC
year,,,,
2020,⋯,⋯,⋯,"144,095"
2021,⋯,⋯,⋯,"512,522"
2022,"1,273,838","550,755",0.0601,"551,086"


,pieces,reports,Δ%,NCMEC
year,,,,
2020,"4,437,853","547,875",-0.2137,"546,704"
2021,"6,696,497","870,319",0.6278,"875,783"
2022,"13,402,885","2,174,319",0.0105,"2,174,548"
